In [45]:
# Each line after the first is either broad, medium, or narrow.
# Also sum for each industry.
# For the first attempt, just ignore the non-national data.
# Descending order of size: Major, Minor, Broad, Detail.
# For NAICS values, all are either two-digit or six digit, right-padded with 0s.
import csv
import re
with open("all_data_M_2015.csv",'r') as f:
    c=csv.reader(f)
    L=list(c)
print(L[0])

['area', 'area_title', 'area_type', 'naics', 'naics_title', 'own_code', 'occ code', 'occ title', 'group', 'tot_emp', 'emp_prse', 'jobs_1000', 'loc_quotient', 'pct_total', 'h_mean', ' a_mean ', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', ' a_pct10 ', ' a_pct25 ', ' a_median ', ' a_pct75 ', ' a_pct90 ', 'annual', 'hourly', '']


In [41]:
d=dict()
def get(i,count):
    return [j[count] for j in i[1:]]
d['naics']= get(l,3)     #naics
d['occ_code']= get(l,6)  #occ code, adjusted name here for consistency
d['group']= get(l,8)     #Group
d['tot_emp']= get(l,9)     #Total employment
d['h_mean']]=get(l,14)    #Mean Hourly Pay
d['a_mean']=get(l,15)    #Mean Annual Pay, adjusted the name here to remove extraneous spaces
d["h_pay_dist"]=get(l,slice(17,22)) #10th, 25th, 50th, 75th, and 90th percentile hourly pay.
d["a_pay_dist"]=get(l,slice(22,27)) #10th, 25th, 50th, 75th, and 90th percentile annual pay.
all_jobs=[[d['naics'][n],d['occ_code'][n],d['group'][n],d['tot_emp'][n],d['h_mean'][n],d['a_mean'][n],
          d["h_pay_dist"][n],d["h_pay_dist"][n]] for n in range(len(d['naics']))]
d.keys()

dict_keys(['naics', 'occ_code', 'group', 'tot_emp', 'h_mean', 'a_mean', 'h_pay_dist', 'a_pay_dist'])

In [74]:
def select(func,universe_list=all_jobs):
    final=list()
    for i in universe_list:
        if func(i):
            final.append(i)
    return final

def match(index,value):
    return lambda x:(x[index]==value)

def access_job(occ_code,universe_dict=d,universe_list=all_jobs):
    code_list=universe_dict['occ_code']
    holder=[]
    for i,q in enumerate(code_list):
        if q==occ_code:
            holder.append(universe_list[i])
    return holder
#access_job("13-2000")

In [75]:
def get_percent(job_num,naics_codes): #job_num should be a string, naics_code a list of relevant naics codes.
    # Also works fine if you just use a single string.
    job_rows=access_job(str(job_num))
    denominator=0
    for row in job_rows:
        if row[0] in naics_codes:
            numerator=int(re.sub(",","",row[3]))
            continue
        else:
            try:
                denominator+=int(re.sub(",","",row[3]))
            except ValueError:
                #print(row[3])
                pass
    return numerator/denominator
get_percent("13-2000","541200") 
# Financial Specialists and Accounting, Tax Preparation, Bookkeeping, and Payroll Services

0.03248307720701991